# Imports

In [11]:
import warnings
warnings.filterwarnings("ignore")

# Load variables
import os
from dotenv import load_dotenv
load_dotenv()

# Snowpark Imports
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Connect to Snowflake

In [12]:
snowflake_connection_cfg = {
    "ACCOUNT": os.getenv('SF_ACCOUNT'),
    "USER": os.getenv('SF_USER'),
    "ROLE": os.getenv('SF_ROLE'),
    "PASSWORD": os.getenv('SF_PASSWORD'),
    "DATABASE": os.getenv('SF_DATABASE'),
    "SCHEMA": os.getenv('SF_SCHEMA'),
    "WAREHOUSE": os.getenv('SF_WAREHOUSE')
}

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

print('Role:     ', session.get_current_role())
print('Warehouse:', session.get_current_warehouse())
print('Database: ', session.get_current_database())
print('Schema:   ', session.get_current_schema())

Role:      "ACCOUNTADMIN"
Warehouse: "COMPUTE_WH"
Database:  "MACHINE_LEARNING"
Schema:    "PUBLIC"


In [132]:
# Create some test data to work with
ai_texts_german = [
    "KI revolutioniert die Geschäftsanalytik, indem sie tiefere Einblicke in Daten bietet.",
    "Unternehmen nutzen KI, um die Analyse und Interpretation komplexer Datensätze zu transformieren.",
    "Mit KI können Unternehmen nun große Datenmengen verstehen, um die Entscheidungsfindung zu verbessern.",
    "Künstliche Intelligenz ist ein Schlüsselwerkzeug für Unternehmen, die ihre Datenanalyse verbessern möchten.",
    "Der Einsatz von KI in Unternehmen hilft dabei, bedeutungsvolle Informationen aus großen Datensätzen zu extrahieren."
]


different_texts_german = [
    "Der große Weiße Hai ist einer der mächtigsten Raubtiere des Ozeans.",
    "Van Goghs Sternennacht stellt die Aussicht aus seinem Zimmer in der Anstalt bei Nacht dar.",
    "Quantencomputing könnte potenziell viele der derzeit verwendeten kryptografischen Systeme brechen.",
    "Die brasilianische Küche ist bekannt für ihre Vielfalt und Reichhaltigkeit, beeinflusst von Europa, Afrika und den amerindischen Kulturen.",
    "Das schnellste Landtier, der Gepard, erreicht Geschwindigkeiten von bis zu 120 km/h."
]


search_text = 'Maschinelles Lernen ist eine unverzichtbare Ressource für Unternehmen, die ihre Dateneinblicke verbessern möchten.'

df = session.create_dataframe(ai_texts_german+different_texts_german, schema=['TEXT'])
df.show()

------------------------------------------------------
|"TEXT"                                              |
------------------------------------------------------
|KI revolutioniert die Geschäftsanalytik, indem ...  |
|Unternehmen nutzen KI, um die Analyse und Inter...  |
|Mit KI können Unternehmen nun große Datenmengen...  |
|Künstliche Intelligenz ist ein Schlüsselwerkzeu...  |
|Der Einsatz von KI in Unternehmen hilft dabei, ...  |
|Der große Weiße Hai ist einer der mächtigsten R...  |
|Van Goghs Sternennacht stellt die Aussicht aus ...  |
|Quantencomputing könnte potenziell viele der de...  |
|Die brasilianische Küche ist bekannt für ihre V...  |
|Das schnellste Landtier, der Gepard, erreicht G...  |
------------------------------------------------------



# Register Embedding Model

In [16]:
# Get the model registry object
from snowflake.ml.registry import Registry
reg = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema()
    )

In [ ]:
# Get the embedding model from Huggingface
# Make sure it fits into a Snowflake warehouse and does not require GPUs
# Otherwise the model must deployed in Snowpark Container Services
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("intfloat/multilingual-e5-small")

In [ ]:
# Register the model to Snowflake
snow_model = reg.log_model(
    model, 
    model_name='multilingual_e5_small', 
    )

In [59]:
# The model can also be used via calling multilingual_e5_small!ENCODE
snow_model.show_functions()

[{'name': 'ENCODE',
  'target_method': 'encode',
  'signature': ModelSignature(
                      inputs=[
                          FeatureSpec(dtype=DataType.STRING, name='TEXT')
                      ],
                      outputs=[
                          FeatureSpec(dtype=DataType.DOUBLE, name='output_feature_0', shape=(384,))
                      ]
                  )}]

# Use Embeddings for Similarity Search

In [133]:
# Create Embeddings from Huggingface Model
embedding_df = snow_model.run(df)
# We have to convert the output of the Huggingface model to Snowflake's Vector Datatype
embedding_df = embedding_df.with_column(
    'EMBEDDING', 
    F.col('"output_feature_0"').cast(T.VectorType(float,384))).drop('"output_feature_0"')
embedding_df.write.save_as_table('EMBEDDED_TEXTS', mode='overwrite')
embedding_df = session.table('EMBEDDED_TEXTS')
embedding_df.show()

-----------------------------------------------------------------------------------------------------------
|"TEXT"                                              |"EMBEDDING"                                         |
-----------------------------------------------------------------------------------------------------------
|Der große Weiße Hai ist einer der mächtigsten R...  |[0.0795096904039383, -0.01898859813809395, -0.0...  |
|Der Einsatz von KI in Unternehmen hilft dabei, ...  |[0.07381673902273178, -0.07341468334197998, -0....  |
|Mit KI können Unternehmen nun große Datenmengen...  |[0.05727183073759079, -0.05122379586100578, -0....  |
|Das schnellste Landtier, der Gepard, erreicht G...  |[0.07121457904577255, -0.02618776448071003, -0....  |
|Künstliche Intelligenz ist ein Schlüsselwerkzeu...  |[0.01078315544873476, -0.036433085799217224, -0...  |
|Quantencomputing könnte potenziell viele der de...  |[0.09990449994802475, -0.04794745147228241, -0....  |
|Van Goghs Sternennacht stel

In [134]:
# Finally we can calculate the distance between all the embeddings and our search vector
closest_texts = embedding_df.with_column(
    'VECTOR_DISTANCE', 
    F.vector_l2_distance(
        F.col('EMBEDDING'), 
        F.call_builtin(
            'MULTILINGUAL_E5_SMALL!ENCODE', 
            F.lit(search_text))['output_feature_0'].cast(T.VectorType(float,384))
    )
).cache_result()

In [135]:
# As we can see, all of the closest texts are AI related like our search vector
closest_texts.order_by('VECTOR_DISTANCE').drop('EMBEDDING').show(max_width=100)

-----------------------------------------------------------------------------------------------------------------------------
|"TEXT"                                                                                                |"VECTOR_DISTANCE"   |
-----------------------------------------------------------------------------------------------------------------------------
|Künstliche Intelligenz ist ein Schlüsselwerkzeug für Unternehmen, die ihre Datenanalyse verbesser...  |0.5223915125274216  |
|KI revolutioniert die Geschäftsanalytik, indem sie tiefere Einblicke in Daten bietet.                 |0.5508320832298432  |
|Mit KI können Unternehmen nun große Datenmengen verstehen, um die Entscheidungsfindung zu verbess...  |0.5517107107937466  |
|Der Einsatz von KI in Unternehmen hilft dabei, bedeutungsvolle Informationen aus großen Datensätz...  |0.5768123622873043  |
|Unternehmen nutzen KI, um die Analyse und Interpretation komplexer Datensätze zu transformieren.      |0.578202268231

# Customize the Model Signature

In [ ]:
from snowflake.ml.model.model_signature import FeatureSpec, DataType, ModelSignature

# In this example the output column will be called EMBEDDING and have a shape of (384,)
model_sig = ModelSignature(
                  inputs=[
                      FeatureSpec(dtype=DataType.STRING, name='TEXT')
                      ],
                      outputs=[
                          FeatureSpec(dtype=DataType.DOUBLE, name='EMBEDDING', shape=(384,))
                      ]
                  )

In [130]:
# Register the model to Snowflake (encode is the model's function we want to call)
snow_model_custom = reg.log_model(
    model, 
    model_name='multilingual_e5_small_custom', 
    signatures={'encode':model_sig}
    )

In [137]:
# Create Embeddings from Huggingface Model
embedding_df = snow_model_custom.run(df)
# We have to convert the output of the Huggingface model to Snowflake's Vector Datatype
embedding_df = embedding_df.with_column('EMBEDDING', F.col('EMBEDDING').cast(T.VectorType(float,384)))
embedding_df.write.save_as_table('EMBEDDED_TEXTS', mode='overwrite')
embedding_df = session.table('EMBEDDED_TEXTS')
embedding_df.show()

-----------------------------------------------------------------------------------------------------------
|"TEXT"                                              |"EMBEDDING"                                         |
-----------------------------------------------------------------------------------------------------------
|Der Einsatz von KI in Unternehmen hilft dabei, ...  |[0.07381673902273178, -0.07341468334197998, -0....  |
|Der große Weiße Hai ist einer der mächtigsten R...  |[0.0795096904039383, -0.01898859813809395, -0.0...  |
|KI revolutioniert die Geschäftsanalytik, indem ...  |[0.07513290643692017, -0.0682922974228859, -0.0...  |
|Künstliche Intelligenz ist ein Schlüsselwerkzeu...  |[0.01078315544873476, -0.036433085799217224, -0...  |
|Quantencomputing könnte potenziell viele der de...  |[0.09990449994802475, -0.04794745147228241, -0....  |
|Van Goghs Sternennacht stellt die Aussicht aus ...  |[0.09676273912191391, -0.0885557308793068, -0.0...  |
|Das schnellste Landtier, de

In [144]:
# Finally we can calculate the distance between all the embeddings and our search vector
closest_texts = embedding_df.with_column(
    'VECTOR_DISTANCE', 
    F.vector_l2_distance(
        F.col('EMBEDDING'), 
        F.call_builtin(
            'MULTILINGUAL_E5_SMALL_CUSTOM!ENCODE', 
            F.lit(search_text))['EMBEDDING'].cast(T.VectorType(float,384))
    )
).cache_result()

# As we can see, all of the closest texts are AI related like our search vector
closest_texts.order_by('VECTOR_DISTANCE').drop('EMBEDDING').show(max_width=100)

-----------------------------------------------------------------------------------------------------------------------------
|"TEXT"                                                                                                |"VECTOR_DISTANCE"   |
-----------------------------------------------------------------------------------------------------------------------------
|Künstliche Intelligenz ist ein Schlüsselwerkzeug für Unternehmen, die ihre Datenanalyse verbesser...  |0.5223915125274216  |
|KI revolutioniert die Geschäftsanalytik, indem sie tiefere Einblicke in Daten bietet.                 |0.5508320832298432  |
|Mit KI können Unternehmen nun große Datenmengen verstehen, um die Entscheidungsfindung zu verbess...  |0.5517107107937466  |
|Der Einsatz von KI in Unternehmen hilft dabei, bedeutungsvolle Informationen aus großen Datensätz...  |0.5768123622873043  |
|Unternehmen nutzen KI, um die Analyse und Interpretation komplexer Datensätze zu transformieren.      |0.578202268231